Remove code for downsampling, not needed

Add this after "define network"

In [ ]:
# Code in this cell is from https://stackoverflow.com/questions/49201236/check-the-total-number-of-parameters-in-a-pytorch-model
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

new version of CNN definition

In [ ]:
class ConvNetScalarLabel(nn.Module):
    def __init__(self, kernel_size = 3, activation_fn = nn.ReLU()):
        super().__init__()

        self.layers = nn.ModuleList()
        self.layers.append(self.create_conv_set(1, 2, kernel_size, activation_fn))
        self.layers.append(self.create_conv_set(2, 2, kernel_size, activation_fn))
        self.layers.append(nn.MaxPool3d(kernel_size = 2))
        self.layers.append(self.create_conv_set(2, 4, kernel_size, activation_fn))
        self.layers.append(self.create_conv_set(4, 4, kernel_size, activation_fn))
        self.layers.append(nn.MaxPool3d(kernel_size = 2))
        self.layers.append(self.create_conv_set(4, 8, kernel_size, activation_fn))
        self.layers.append(self.create_conv_set(8, 8, kernel_size, activation_fn))
        self.layers.append(nn.MaxPool3d(kernel_size = 2))
        self.layers.append(self.create_conv_set(8, 32, kernel_size, activation_fn))
        self.layers.append(self.create_conv_set(32, 32, kernel_size, activation_fn))
        self.layers.append(nn.MaxPool3d(kernel_size = 2))
        self.layers.append(self.create_conv_set(32, 128, kernel_size, activation_fn))
        self.layers.append(self.create_conv_set(128, 128, kernel_size, activation_fn))
        self.layers.append(nn.MaxPool3d(kernel_size = 2))
        self.layers.append(self.create_conv_set(128, 256, kernel_size, activation_fn))
        self.layers.append(self.create_conv_set(256, 256, kernel_size, activation_fn))
        self.layers.append(nn.MaxPool3d(kernel_size = 8))

        self.linear_1 = nn.Linear(256, 16)
        self.linear_2 = nn.Linear(16, 1)

    def create_conv_set(self, in_channels, out_channels, kernel_size, activation_fn):
        return nn.Sequential(
            nn.Conv3d(in_channels = in_channels, out_channels = out_channels, kernel_size = kernel_size, padding = 'same'),
            nn.BatchNorm3d(num_features = out_channels),
            activation_fn
        )

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = torch.squeeze(x)
        x = self.linear_1(x)
        x = self.linear_2(x)
        return torch.squeeze(x)

Separate out network definitions to new file